In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [5]:
data_dir = "4B-2D-Uni"

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Classes: {train_dataset.classes}")


Classes: ['Stable', 'Unstable']


In [79]:
import torch
import torchvision.models as models
import torch.nn as nn
import timm
from efficientnet_pytorch import EfficientNet
from transformers import AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_feature_extractor(model_name):
    print(f"Attempting to load model: {model_name}")

    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1])

    elif model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1]) 
        
    elif model_name == "alexnet":
        model = models.alexnet(pretrained=True)
        feature_extractor = model.features  
    
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        feature_extractor = model.features
    
    elif model_name == "densenet121":
        model = models.densenet121(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1])
    
    elif model_name == "densenet169":
        model = models.densenet169(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1])
    
    elif model_name == "densenet201":
        model = models.densenet201(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1])
    
    elif model_name == "googlenet":
        model = models.googlenet(pretrained=True)
        feature_extractor = nn.Sequential(*list(model.children())[:-1])
    
    elif model_name == "inception_v3":
        model = models.inception_v3(pretrained=True, aux_logits=True)
        model.fc = nn.Identity()
        feature_extractor = model
    
    # EfficientNet models
    elif model_name in ["efficientnet-b0", "efficientnet-b1", "efficientnet-b2", 
                         "efficientnet-b3", "efficientnet-b4", "efficientnet-b5", 
                         "efficientnet-b6", "efficientnet-b7"]:
        model = EfficientNet.from_pretrained(model_name)
        feature_extractor = model
    
    # EfficientNetV2 models using timm
    elif model_name in ["tf_efficientnetv2_b0", "efficientnetv2_b1", "efficientnetv2_s"]:
        # EfficientNetV2 from timm
        model = timm.create_model(model_name, pretrained=True)
        feature_extractor = model

    # ConvNeXt models
    elif model_name in ["convnext_tiny", "convnext_small", "convnext_base", "convnext_large"]:
        # Load ConvNeXt from timm
        model = timm.create_model(model_name, pretrained=True)
        feature_extractor = model


    # ConvNeXtV2 models
    elif model_name in ["convnextv2_tiny", "convnextv2_small", "convnextv2_base", "convnextv2_large"]:
        # Load ConvNeXtV2 from timm
        model = timm.create_model(model_name, pretrained=True)
        feature_extractor = model

    # DEIT models
    elif model_name in ["deit_tiny_patch16_224", "deit_small_patch16_224", "deit_base_patch16_224"]:
        # Load DEIT from timm
        model = timm.create_model(model_name, pretrained=True)
        feature_extractor = model

    # ViT models
    elif model_name in ["vit_tiny_patch16_224", "vit_small_patch16_224", "vit_base_patch16_224", "vit_large_patch16_224"]:
        # Load ViT from timm
        model = timm.create_model(model_name, pretrained=True)
        feature_extractor = model

    # DINOv2 models
    elif model_name == "dino_vits":
        model = AutoModel.from_pretrained("facebook/dinov2-small").to(device)
        feature_extractor = model


    
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    feature_extractor = feature_extractor.to(device)
    feature_extractor.eval()
    
    return feature_extractor

# Example of calling the function with EfficientNetV2-S (from timm) with random weights or pretrained weights
#feature_extractor = get_feature_extractor("tf_efficientnetv2_b0")  # Use pretrained EfficientNetV2-B0
#feature_extractor = get_feature_extractor("convnext_tiny")  # ConvNeXt-Tiny from timm

#feature_extractor = get_feature_extractor("convnextv2_tiny")
#feature_extractor = get_feature_extractor("deit_base_patch16_224")
#feature_extractor = get_feature_extractor("vit_base_patch16_224")
#feature_extractor = get_feature_extractor("dino_vits")
#feature_extractor = get_feature_extractor("efficientnet-b1")
#feature_extractor = get_feature_extractor("resnet18")
feature_extractor = get_feature_extractor("densenet121")

print("Feature extractor initialized.")


C:\tools\Anaconda\envs\FPY_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\tools\Anaconda\envs\FPY_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\ajeet/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


Attempting to load model: densenet121


100%|█████████████████████████████████████████████████████████████████████████████| 30.8M/30.8M [00:05<00:00, 5.59MB/s]

Feature extractor initialized.


In [81]:
def extract_features(data_loader, model):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, label in data_loader:
            images = images.to(device)
            output = model(images)  # Extract features
            output = output.view(output.size(0), -1)  # Flatten
            features.append(output.cpu().numpy())
            labels.append(label.numpy())

    return np.vstack(features), np.hstack(labels)

# Extract features from datasets
train_features, train_labels = extract_features(train_loader, feature_extractor)
val_features, val_labels = extract_features(val_loader, feature_extractor)
test_features, test_labels = extract_features(test_loader, feature_extractor)

print("Feature extraction complete.")


Feature extraction complete.


In [15]:
#ONLY FOR DINOv2
def extract_features(data_loader, model):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, label in data_loader:
            images = images.to(device)

            
            output = model(images)

            output = output.pooler_output  

            features.append(output.cpu().numpy())
            labels.append(label.numpy())

    return np.vstack(features), np.hstack(labels)

train_features, train_labels = extract_features(train_loader, feature_extractor)
val_features, val_labels = extract_features(val_loader, feature_extractor)
test_features, test_labels = extract_features(test_loader, feature_extractor)

print("Feature extraction complete.")


Feature extraction complete.


In [83]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

# Train
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(train_features, train_labels)

# Validate
val_preds = svm.predict(val_features)
val_acc = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_acc:.4f}")


Validation Accuracy: 0.8950


In [85]:

test_preds = svm.predict(test_features)
test_acc = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.8800


In [87]:
new_test_dir = "10B-2D-Uni/test"

# Load the new test dataset
new_test_dataset = datasets.ImageFolder(new_test_dir, transform=transform)
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

print(f"New Test Classes: {new_test_dataset.classes}")

new_test_features, new_test_labels = extract_features(new_test_loader, feature_extractor)

new_test_features = scaler.transform(new_test_features)

new_test_preds = svm.predict(new_test_features)

new_test_acc = accuracy_score(new_test_labels, new_test_preds)
print(f"New Test Accuracy: {new_test_acc:.4f}")



New Test Classes: ['Stable', 'Unstable']
New Test Accuracy: 0.6100
